This pipeline is designed to train the model in an automated way.
First, we will define a config with all the specifications. This dictionary is vital to keep track of the models we have used.
Then run the model and extract the metrics that we need, to evaluate the training.

In [1]:
#import config from file
import json

with open("config.json", 'r') as f:
    config = json.load(f)

print(config)

{'model': 'sequence', 'submodel': 'implicit', 'loss': 'pointwise', 'representation': 'pooling', 'embedding_dim': 32, 'n_iter': 10, 'batch_size': 256, 'l2': 0.0, 'lr': 0.01, 'optim': None, 'use_cuda': False, 'sparse': False, 'random_state': None, 'num_negative_samples': 5, 'dataset': 'Movielens', 'size': '100K'}


In [2]:
# or define it manually

config = {
    # About the model
    "model": "sequence",    # sequence or factorizer
    "submodel": "implicit",    # implicit or explicit
    "loss": "bpr",    # one of ‘pointwise’, ‘bpr’, ‘hinge’, ‘adaptive_hinge’ or 'regression', 'poisson','logistic'
    "representation": "pooling",    # for sequence one of ‘pooling’, ‘cnn’, ‘lstm’, ‘mixture’, for factorizer always NONE !!!
    "embedding_dim": 32, 
    "n_iter": 10,
    "batch_size": 256,
    "l2": 0.0,
    "lr": 0.01,
    "optim": None,
    "use_cuda": False, 
    "sparse": False, 
    "random_state": None, 
    "num_negative_samples": 5,

    # About the database
    "dataset": "Movielens",    # Movielens, Synthetic or Goodbooks
    "size": "100K",
    # synthetic has a lot of different parameters but I'm not sure we are going to use it
}

In [3]:
from settings_definition import *

In [4]:
def evaluate_model(model, test):
    """This function evaluates the metric agreed by the team.
    At the moment we have not decided which ones so I just pick 2 as an example."""
    from spotlight.evaluation import mrr_score, precision_recall_score, rmse_score, sequence_mrr_score, sequence_precision_recall_score
    eval = {"mrr": mrr_score(model, test), "precision_recall": precision_recall_score(model, test), #"rmse": rmse_score(model, test),
            #"sequence_mrr":sequence_mrr_score(model, test), "sequence_precision_recall": sequence_precision_recall_score(model, test)
           }
    return eval

In [5]:
import os
os.chdir("../spotlight")
# for me, it only works if I'm in the spotlight directory

In [6]:
# Execution

from spotlight.cross_validation import random_train_test_split

dataset = define_dataset(config)
train, test = random_train_test_split(dataset)
if config["model"] == "sequence":
    train, test = train.to_sequence(), test
model = define_model(config)
model.fit(train)

eval = evaluate_model(model, test)
eval

{'mrr': array([0.01228041, 0.00125578, 0.00147937, 0.14467787, 0.01072238,
        0.02859211, 0.01858905, 0.07011403, 0.05354199, 0.00529649,
        0.00957675, 0.01455785, 0.00700449, 0.02547817, 0.06192295,
        0.02965171, 0.00309258, 0.00666501, 0.00358319, 0.048972  ,
        0.00458405, 0.01098694, 0.0123811 , 0.03543071, 0.00291969,
        0.00217596, 0.00234832, 0.01485055, 0.00104045, 0.01155209,
        0.00241331, 0.00206587, 0.00114046, 0.00174643, 0.00065018,
        0.00062688, 0.00933089, 0.00744874, 0.00174836, 0.00077115,
        0.00125957, 0.03700024, 0.0134947 , 0.01463049, 0.00248535,
        0.00277369, 0.00121213, 0.01724316, 0.01769294, 0.11041083,
        0.00312768, 0.00533765, 0.00148025, 0.00176223, 0.03405282,
        0.03548763, 0.00359012, 0.02149908, 0.01792763, 0.02462688,
        0.00220808, 0.02616569, 0.00152486, 0.01284009, 0.01355796,
        0.00214219, 0.00173221, 0.00206286, 0.00662261, 0.03041652,
        0.00696928, 0.01707941, 0.0058466

In [13]:
import torch

torch.save(model, "../pipeline/trained_model")